# Introduction

- <span style="color:lightgreen">***Generic**: Every programming language has tools for effectively handling the duplication of concepts. In Rust, one such tool is **generics: abstract stand-ins for concrete types or other properties**. Functions can take parameters of some generic type, e.g. `Option<T>`, `Vec<T>`..., instead of a concrete type like `i32` or `String`*</span>

- <span style="color:lightgreen">***Traits: We can use traits to define behavior in a generic way**. You can combine traits with generic types to constrain a generic type to accept only those types that have a particular behavior, as opposed to just any type.*</span>

- <span style="color:lightgreen">***Lifetimes: a variety of generics that give the compiler information about how references relate to each other.***. *Lifetimes allow us to give the compiler enough information about borrowed values so that it can ensure references will be valid in more situations than it could without our help.*</span>


# Generic Data Types

<span style="color:skyblue">***We use generics to create definitions for items like function signatures or structs, which we can then use with many different concrete data types.***</span>

## In Function Definitions

Instead of writing 2 functions that differ only in the type they operate on:
- `fn largest_i32(list: &[i32]) -> &i32`, and
- `fn largest_char(list: &[char]) -> &char`
We can define a function that takes in a generic type `T` with 
```rust
fn largest<T>(list: &[T]) -> &T {
    // snip
}
```
like below


In [2]:
fn largest<T>(list: &[T]) -> &T {
    let mut largest = &list[0];

    for item in list {
        if item > largest {
            largest = item;
        }
    }

    largest
}

fn main() {
    let number_list = vec![34, 50, 25, 100, 65];

    let result = largest(&number_list);
    println!("The largest number is {}", result);

    let char_list = vec!['y', 'm', 'a', 'q'];

    let result = largest(&char_list);
    println!("The largest char is {}", result);
}

Error: binary operation `>` cannot be applied to type `&T`

<span style="color:orange">***We got an error that mentions the `std::cmp::PartialOrd` traits, and this error states that the body of `largest` won’t work for all possible types that `T` could be, but only those that implement the `PartialOrd` trait.***</span>

## In Struct Definitions

Below we defines `Point<T>` to hold values of both `int` and `float`, but `x` and `y` must be of the same type:

In [8]:
#[derive(Debug)]
struct Point<T> {
    x: T,
    y: T,
}

fn main() {
    let integer = Point { x: 5, y: 10 };
    println!("integer point = {:?}", integer);
    let float = Point { x: 1.0, y: 4.0 };
    println!("float point = {:?}", float);
}

main()

integer point = Point { x: 5, y: 10 }
float point = Point { x: 1.0, y: 4.0 }


()

We can make a `Point<T, U>` struct where `x` and `y` have different types like below

In [11]:
#[derive(Debug)]
struct Point<T, U> {
    x: T,
    y: U,
}

fn main() {
    let both_integer = Point { x: 5, y: 10 };
    println!("both int point = {:?}", both_integer);
    let both_float = Point { x: 1.0, y: 4.0 };
    println!("both float point = {:?}", both_float);
    let integer_and_float = Point { x: 5, y: 4.0 };
    println!("int & float point = {:?}", integer_and_float);
}

main()

both int point = Point { x: 5, y: 10 }
both float point = Point { x: 1.0, y: 4.0 }
int & float point = Point { x: 5, y: 4.0 }


()

## In Enum Definitions

Similar to structs, we can define enums with generic types, e.g. the `Option<T>`
```rust
enum Option<T> {
    Some(T),
    None,
}
```
and the enum `Result<T, E>`
```rust
Result<T, E> {
    Ok(T),
    Err(E),
}
```
that has 2 two variants: `Ok`, which holds a value of type `T`, and `Err`, which holds a value of type `E`.

## In Method Definitions

<span style="color:skyblue">***We can implement methods on structs and enums and use generic types in their definitions, too.***</span>

In [15]:
struct Point<T> {
    x: T,
    y: T,
}

impl<T> Point<T> {
    fn x(&self) -> &T {
        &self.x
    }
}

fn main() {
    let p = Point { x: 5, y: 10 };
    println!("p.x = {}", p.x());
    let p2 = Point { x: 1.1, y: 10.3 };
    println!("p2.x = {}", p2.x());
}

main()

p.x = 5
p2.x = 1.1


()

# Traits: Defining Shared Behavior

# Lifetimes for Validating References